In [1]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from IPython.display import clear_output


In [2]:
# Mapping from folder names to numeric labels
Objects = {
    "Wrench": 0,
    "Screwdriver": 1,
    "Hammer": 2,
    "CombWrench": 3
}

In [3]:
%%time
# Merge all the csv files in Data: this takes a while depending on the amount of csv files
def merge_csv_files(folder_path, iteration):
    # List to hold DataFrames
    dfs = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        f_p = folder_path + "/" + filename
        for fn in os.listdir(f_p):
            if fn.endswith('.csv'):
                # Construct full file path
                file_path = f_p + "/" + "batch_" + f"{iteration}" + ".csv"
                print(file_path)
                # Read the CSV file and append to the list
                df = pd.read_csv(file_path)
                dfs.append(df)
            break

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dfs, ignore_index=True)

    return merged_df

CPU times: total: 0 ns
Wall time: 0 ns


In [49]:
def preprocessing(df):
    # Split between labels, and pixel values
    # put labels into y_train variable
    Y = df["Label"]
    # print(Y.head())
    # Drop 'label' column
    X = df.drop(labels = ["Label"],axis = 1)
    # print(X.head())
    # Normalize the data: working with grayscale images so this should be sufficient
    X /= 255
    # print(X.head())
    # Label encoding of the labels
    Y_categorical = to_categorical(Y, num_classes = 4)
    # print(Y_categorical)
    # Reshape
    X = X.values.reshape(-1,512,512,1)
    print("X shape: ",X.shape)
    # Data split into train and test
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y_categorical, test_size = 0.2)
    print("x_train shape",X_train.shape)
    print("x_test shape",X_val.shape)
    print("y_train shape",Y_train.shape)
    print("y_test shape",Y_val.shape)
    return X_train, X_val, Y_train, Y_val
def preprocessing_kf(df, target_column='target'):
    """
    Preprocesses the given DataFrame by scaling numerical features and encoding categorical ones.
    Arguments:
        df: pandas DataFrame containing the dataset to preprocess.
        target_column: string, the name of the column in df that is the target variable.
    Returns:
        X: Features matrix.
        Y: Target variable array.
    """

    # Separate the features and the target from the DataFrame
    Y = df[target_column]
    X = df.drop(columns=[target_column])
    
    # Define which columns are numeric and which are categorical
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object', 'bool']).columns
    
    # Create transformers for numeric and categorical data
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine transformers into a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    # Apply the transformations to the feature data
    X_preprocessed = preprocessor.fit_transform(X)
    
    # Optionally, convert the preprocessed data back to a DataFrame for further use
    # This step is optional and can be omitted if the rest of your pipeline can work with NumPy arrays directly
    # X_preprocessed = pd.DataFrame(X_preprocessed, columns=feature_names)

    return X_preprocessed, Y.values

In [53]:
# CNN imports
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow.keras.utils import to_categorical
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [50]:
model = Sequential()

# Input and first convolutional block with normalization and increased L2 regularization
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='Same', 
                 activation='relu', input_shape=(512,512,1),
                 kernel_regularizer=l2(0.01)))  # Increased L2 regularization
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))  # Increased dropout

# Second convolutional block with increased L2 regularization
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='Same', 
                 activation='relu',
                 kernel_regularizer=l2(0.01)))  # Increased L2 regularization
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.4))  # Increased dropout

# Fully connected layers with L2 regularization
model.add(Flatten())
model.add(Dense(256, activation="relu", kernel_regularizer=l2(0.001)))  # Reduced number of neurons and added L2 regularization
model.add(Dropout(0.5))  # Kept the same dropout rate here
model.add(Dense(4, activation="softmax"))  # Output layer remains the same

# Define the optimizer with a reduced learning rate
optimizer = Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999)

# Compile the model with the same loss function and metrics
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [16]:
def runModel(m, e, b_s, X_t, Y_t, X_v,Y_v, early_stopping):
    epochs = e
    batch_size = b_s
    # Fit the model
    history = m.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val,Y_val), callbacks=[early_stopping])

In [54]:
# Create a training loop
folder_path = 'Data/Batches'
pairing = 1

# Initialize an empty dataframe to accumulate all data
big_df = pd.DataFrame()

for i in range(1, 11, 2):
    for j in range(pairing):
        if (i+j == 11):
            break
        index = i + j
        # retrieve data
        df = merge_csv_files(folder_path, index)
        # big_df = pd.concat([big_df, df], ignore_index=True)
        # Clear the output after each iteration
        # clear_output(wait=True)
    
        # train the model
        X_train, X_val, Y_train, Y_val = preprocessing(df)
        
        runModel(model, 10, 9, X_train, X_val, Y_train, Y_val, early_stopping)
        # Reset dataframe
        # big_df = pd.DataFrame()

Data/Batches/CombWrench/batch_1.csv
Data/Batches/Hammer/batch_1.csv
Data/Batches/Screwdriver/batch_1.csv
Data/Batches/Wrench/batch_1.csv
Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 262144), dtype=float32). Expected shape (None, 512, 512, 1), but input has incompatible shape (None, 262144)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 262144), dtype=float32)
  • training=True
  • mask=None

In [11]:
# Create a training loop
folder_path = 'Data/Batches'
pairing = 5

# Initialize an empty dataframe to accumulate all data
big_df = pd.DataFrame()

for i in range(1, 12):
    # retrieve data
    df = merge_csv_files(folder_path, i)
    big_df = pd.concat([big_df, df], ignore_index=True)
    # Clear the output after each iteration
    # clear_output(wait=True)
    
# train the model
X_train, X_val, Y_train, Y_val = preprocessing(big_df)
runModel(model, 20, 72, X_train, X_val, Y_train, Y_val)
# Reset dataframe
big_df = pd.DataFrame()

Data/Batches/CombWrench/batch_1.csv
Data/Batches/Hammer/batch_1.csv
Data/Batches/Screwdriver/batch_1.csv
Data/Batches/Wrench/batch_1.csv
Data/Batches/CombWrench/batch_2.csv
Data/Batches/Hammer/batch_2.csv
Data/Batches/Screwdriver/batch_2.csv
Data/Batches/Wrench/batch_2.csv
Data/Batches/CombWrench/batch_3.csv
Data/Batches/Hammer/batch_3.csv
Data/Batches/Screwdriver/batch_3.csv
Data/Batches/Wrench/batch_3.csv
Data/Batches/CombWrench/batch_4.csv
Data/Batches/Hammer/batch_4.csv
Data/Batches/Screwdriver/batch_4.csv
Data/Batches/Wrench/batch_4.csv
Data/Batches/CombWrench/batch_5.csv
Data/Batches/Hammer/batch_5.csv
Data/Batches/Screwdriver/batch_5.csv
Data/Batches/Wrench/batch_5.csv
Data/Batches/CombWrench/batch_6.csv
Data/Batches/Hammer/batch_6.csv
Data/Batches/Screwdriver/batch_6.csv
Data/Batches/Wrench/batch_6.csv
Data/Batches/CombWrench/batch_7.csv
Data/Batches/Hammer/batch_7.csv
Data/Batches/Screwdriver/batch_7.csv
Data/Batches/Wrench/batch_7.csv
Data/Batches/CombWrench/batch_8.csv
Data/

KeyboardInterrupt: 

In [ ]:
# Save the current model
model.save('my_model.keras')